In [ ]:
!git clone https://github.com/strivn/idl-project

Cloning into 'idl-project'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 417 (delta 131), reused 94 (delta 56), pack-reused 215 (from 1)
Receiving objects: 100% (417/417), 15.78 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (244/244), done.


In [ ]:
# Append paths for the src folder
import sys
import os
sys.path.append("/content/idl-project")

# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'idl-project')))

# Additional imports
from src.utils import *


Created directory: .cache/


In [ ]:
!pip install datasets transformers peft accelerate bitsandbytes rouge-score sentencepiece --quiet


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00

In [ ]:
from src.model import load_fo_model, load_ba_model
from src.data import load_flan_dataset, load_summarization_datasets

Using device: cuda


In [ ]:
import torch
import wandb

from datasets import Dataset, load_dataset
from peft import (LoraConfig, PeftModel, get_peft_model,
                  prepare_model_for_kbit_training)
from tqdm.auto import tqdm
from transformers import (AutoTokenizer, BitsAndBytesConfig,
                          DataCollatorForLanguageModeling, GPTNeoXForCausalLM,
                          Trainer, TrainingArguments, TrainerCallback)

In [ ]:
from src.data import load_summarization_datasets

rawdataset = load_dataset(
    "Open-Orca/FLAN",
    data_files={"flan_zsopt_data": "flan_zsopt_data/*.parquet"}
)
dataset = rawdataset["flan_zsopt_data"].select(range(50000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.83k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/83 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/83 [00:00<?, ?it/s]

part.0.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

...

part.81.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

part.82.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

part.9.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

Generating flan_zsopt_data split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
import os
print(f"📁 Current working directory: {os.getcwd()}")

print(f"✓ Directory contains 'idl-project'")

OUTPUT_DIR = "output/"
from notebooks.finetune_lora_config import *

📁 Current working directory: /content
✓ Directory contains 'idl-project'


In [ ]:
import wandb
wandb.init(
    project="fifthbafo reinforce",
    entity="11785_finetuning",
    name="xuanyi-rein-test",
    group="xuanyishen",
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xuanyis (xuanyis-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/models/fo_fifth"

fo_model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
fo_tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

In [ ]:
model_dir = "/content/drive/MyDrive/models/ba_fifth"
from transformers import AutoModelForCausalLM, AutoTokenizer

ba_model = AutoModelForCausalLM.from_pretrained(model_dir)
ba_tokenizer = AutoTokenizer.from_pretrained(model_dir)


# unsuoervised finetune on foward model.

Use backward finetune as a feedback

In [ ]:
from torch.utils.data import Dataset, DataLoader


class InputOnlyDataset(Dataset):
    def __init__(self, inputs, tokenizer, max_length=1024):
        self.inputs = inputs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {"input_text": self.inputs[idx]}

In [ ]:

batch_size = 4
input_texts = dataset["inputs"]
dataset = InputOnlyDataset(input_texts, fo_tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
fo_tokenizer.pad_token = fo_tokenizer.eos_token
fo_tokenizer.pad_token_id = fo_tokenizer.eos_token_id
fo_model.config.pad_token_id = fo_tokenizer.eos_token_id

ba_tokenizer.pad_token = ba_tokenizer.eos_token
ba_tokenizer.pad_token_id = ba_tokenizer.eos_token_id

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def score_answer_with_ba(input_text, generated_text, ba_model, ba_tokenizer, device="cuda", debug=False):


    context = generated_text
    target = input_text

    context_ids = ba_tokenizer.encode(context, return_tensors="pt").to(device)
    target_ids = ba_tokenizer.encode(target, return_tensors="pt").to(device)

    context_len = context_ids.shape[1]
    target_len = target_ids.shape[1]

    context_ids = torch.flip(context_ids, [1])
    target_ids = torch.flip(target_ids, [1])

    input_ids = torch.cat([context_ids, target_ids], dim=1)

    with torch.no_grad():
        outputs = ba_model(input_ids)
        logits = outputs.logits

    token_log_probs = []
    for i in range(context_len - 1, context_len + target_len - 1):
        next_token_logits = logits[0, i, :]
        next_token_id = input_ids[0, i + 1].item()

        prob = F.softmax(next_token_logits, dim=0)[next_token_id].item()
        log_prob = np.log(prob)
        token_log_probs.append(log_prob)

        if debug:
            token_str = ba_tokenizer.decode([next_token_id])
            print(f"{token_str}\tlog_prob={log_prob:.4f}")

    sequence_log_prob = sum(token_log_probs)
    normalized_log_prob = sequence_log_prob / len(token_log_probs)
    perplexity = np.exp(-normalized_log_prob)

    return {
        "sequence_log_prob": sequence_log_prob,
        "normalized_log_prob": normalized_log_prob,
        "perplexity": perplexity
    }

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import get_scheduler
from src.utils import calculate_score

def train_unsupervised_foba(
    fo_model,
    ba_model,
    fo_tokenizer,
    ba_tokenizer,
    dataloader,
    num_epochs=1,
    lr=5e-5,
    max_new_tokens=64,
    n_samples_per_input=3,
    device="cuda"
):
    fo_model.train().to(device)
    ba_model.eval().to(device)
    optimizer = torch.optim.AdamW(fo_model.parameters(), lr=lr)
    num_training_steps = len(dataloader) * num_epochs
    lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    for epoch in range(num_epochs):
        progress = tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False)
        for step, batch in enumerate(progress):
            input_texts = batch["input_text"]
            losses = []
            for input_text in input_texts:
                inputs = fo_tokenizer(input_text, return_tensors="pt").to(device)
                generated_ids = fo_model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=fo_tokenizer.eos_token_id,
                    num_return_sequences=n_samples_per_input,
                )
                generated_texts = fo_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

                rewards = []
                for gen_text in generated_texts:
                    with torch.no_grad():
                        score_info = score_answer_with_ba(
                            input_text=input_text,
                            generated_text=gen_text,
                            ba_model=ba_model,
                            ba_tokenizer=ba_tokenizer,
                            device=device,
                            debug=False
                        )
                        rewards.append(score_info["normalized_log_prob"])

                rewards = torch.tensor(rewards, device=device)  # (n_samples, )

                # 3. baseline: 取平均分
                baseline = rewards.mean()

                # 4. 找到得分最高的 generated_text
                best_idx = rewards.argmax().item()
                best_generated_text = generated_texts[best_idx]
                best_reward = rewards[best_idx]

                # 5. 重新tokenize最佳回答
                full_input = input_text + fo_tokenizer.eos_token + best_generated_text
                tokenized = fo_tokenizer(full_input, return_tensors="pt").to(device)
                input_ids = tokenized["input_ids"]

                outputs = fo_model(input_ids)
                logits = outputs.logits[:, :-1, :]
                labels = input_ids[:, 1:]

                log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
                chosen_log_probs = log_probs.gather(2, labels.unsqueeze(2)).squeeze(-1)
                log_prob_mean = chosen_log_probs.mean()

                loss = -(best_reward - baseline) * log_prob_mean
                losses.append(loss)

            if losses:
                batch_loss = torch.stack(losses).mean()
                batch_loss.backward()
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress.set_postfix(loss=batch_loss.item())

In [ ]:
train_unsupervised_foba(
    fo_model=fo_model,
    ba_model=ba_model,
    fo_tokenizer=fo_tokenizer,
    ba_tokenizer=ba_tokenizer,
    dataloader=dataloader,
    num_epochs=1,
    lr=5e-5,
    max_new_tokens=64,
    n_samples_per_input=3,
    device="cuda"
)

Epoch 1:   0%|          | 24/12500 [01:23<11:36:18,  3.35s/it, loss=0.102]This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


KeyboardInterrupt: 

In [ ]:
save_path = "/content/drive/MyDrive/models/fo_unsupv2"

fo_model.save_pretrained(save_path)
fo_tokenizer.save_pretrained(save_path)